In [1]:
from DigitsModule import *

Using TensorFlow backend.
W1216 00:50:39.447544  3580 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1216 00:50:39.479548  3580 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1216 00:50:39.499544  3580 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1216 00:50:39.503540  3580 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:148: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W1216 00:50:39.51554

In [9]:
img = io.imread("test.tif")
rec_cell(img)

C:\ProgramData\Anaconda3\lib\site-packages\skimage\feature\_hog.py:150: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15. To supress this message specify explicitly the normalization method.
  skimage_deprecation)
C:\ProgramData\Anaconda3\lib\site-packages\skimage\feature\_hog.py:150: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15. To supress this message specify explicitly the normalization method.
  skimage_deprecation)
C:\ProgramData\Anaconda3\lib\site-packages\skimage\feature\_hog.py:150: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15. To supress this message specify explicitly the normalization method.
  skimage_deprecation)


'10'

In [42]:
def rec_cell(img):
    digit_strv1 = ""
    digit_strv2 = ""
    img = rgb2gray(img)
    NotThin = (np.amax(thin(img)-img))
    
    imgv1 = np.copy(img)
    imgv1 = erosion(imgv1)
    imgv1 = binary_dilation(imgv1)
    digitsv1 = digits_locv1(imgv1)
    #show_images(digitsv1)
    
    imgv2 = np.copy(img)
    imgv2 = rgb2gray(imgv2)
    imgv2 = erosion(imgv2)
    digitsv2 = digits_locv1(imgv2)
    #show_images(digitsv2)
    for dig in digitsv1:
        digit_strv1 += str(predict_img(dig))
    
    for dig in digitsv2:
        digit_strv2 += str(predict_img(dig))
    
    if(len(digit_strv1)!= "" and NotThin != 0):
        return digit_strv1
    else:
        return digit_strv2

In [36]:
def digits_locv1(img,X=28,Y=28):
    """
    Localization of the digits , it separates each digit into a fixed size output
    Arugments :
    -- img : numpy array
    Returns
    -- digits :  Array of fixed size matrices for each digit .
    """
    X = int(X/2)
    Y = int(Y/2)
    img = rgb2gray(img) 
    img = rotate(img,270,resize=True)
    img_hist = histogram(img, nbins=2)
    # Checking the image`s background must be black and digits be white
    # Negative Transformation in case of white (objects) is more than black (background)
    if ( img_hist[0][0] < img_hist[0][1] ):
        img = 1 - img 
    
    digits = []
    # Find contours for each digit has its own contour
    contours = find_contours(img, 0.7,fully_connected='high',positive_orientation='high')
    for n, contour in enumerate(contours):
        
        #print(len(contour))
        if(len(contour) < 50) :
            continue
        Ymax = np.amax(contour[:, 0])
        Ymin = np.amin(contour[:, 0])
        Xmax = np.amax(contour[:, 1])
        Xmin = np.amin(contour[:, 1])
        digit_seg = ([img[int(Ymin): int(Ymax)+1, int(Xmin): int(Xmax)+1]])
        digit = np.zeros([X*2,Y*2])
        h,w = np.array(digit_seg[0]).shape
        if(h > 28 or w>28):
            continue
        digit[X-int((h+1)/2):X+int(h/2) ,Y-int((w+1)/2):Y+int(w/2) ,  ] = digit_seg[0]
        digit = rotate(digit,90,resize=True)
        digit = erosion(digit)
        digit = dilation(digit)
        digits.append(digit)
        
    return digits